In [ ]:
!pip install gensim

In [ ]:
!pip install nltk

In [ ]:
# !pip install tensorflow
!pip install tensorflow==2.12.0


In [ ]:
# !pip install --upgrade tensorflow transformers
!pip install transformers==4.31.0

In [ ]:
import pandas as pd
import numpy as np
import re
import string
import gensim
from gensim.parsing.preprocessing import remove_stopwords
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.ensemble import GradientBoostingClassifier

import tensorflow as tf
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
from transformers import BertTokenizer, TFBertForSequenceClassification, DistilBertTokenizer, TFDistilBertForSequenceClassification




In [ ]:
# import tensorflow.compat.v1 as tff
# tff.disable_v2_behavior()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('punkt_tab')

In [ ]:
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
bully_df = pd.read_csv('./drive/MyDrive/datasets/cyberbullying_tweets.csv')

In [ ]:
def cleantxt(txt):
    stpw = stopwords.words('english')

    stpw.extend(['www','http','utc'])
    stpw = set(stpw)
    txt = re.sub(r"\n", " ", txt)
    txt = re.sub("[\<\[].*?[\>\]]", " ", txt)
    txt = txt.lower()
    txt = re.sub(r"[^a-zA-Z ]", " ", txt)
    txt = re.sub(r"\b\w{1,3}\b", " ",txt)
    txt = " ".join([x for x in txt.split() if x not in stpw])
    return txt

In [ ]:
bully_df = bully_df.rename(columns={'tweet_text':'tweet','cyberbullying_type':'type'})

In [ ]:
bully_df['tweet'] = bully_df['tweet'].apply(lambda x: re.sub(r'@\w+', '', x).strip())

In [ ]:
bully_df['tweet'] = bully_df['tweet'].apply(cleantxt)

In [ ]:
bully_df['tweet'] = bully_df['tweet'].apply(lambda x: x.lower())


In [ ]:
bully_df['tweet'] = bully_df['tweet'].apply(lambda x: ' '.join([w for w in x.split() if w != 'rt']))


In [ ]:
bully_df.tail(10)

In [ ]:
# lemmatizer = WordNetLemmatizer()

# def lemmatize_sentence(sentence):
#     tokens = nltk.word_tokenize(sentence)
#     lemmatized_words = [lemmatizer.lemmatize(word, pos='v') for word in tokens]
#     return ' '.join(lemmatized_words)

In [ ]:
# bully_df['tweet'] = bully_df['tweet'].apply(lemmatize_sentence)

In [ ]:
from wordcloud import WordCloud
negative_words = ' '.join([text for text in bully_df['tweet'][bully_df['type'] != 'not_cyberbullying']])
wordcloud = WordCloud(width=800, height=500,
random_state=21, max_font_size=110).generate(negative_words)
plt.figure(figsize=(10, 7))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.show()

In [ ]:
# bully_df['tweetsplit'] = bully_df['tweet'].apply(lambda x: x.split())
# bully_df['tweet'] = bully_df['tweet'].apply(lambda x: ([w for w in x if len(w)>=3]))
# bully_df['tweetcnn'] = bully_df['tweet'].apply(lambda x: ' '.)

In [ ]:
# gender religion age ethnicity other_cyberbullying
not_bully_df = bully_df[bully_df['type'] == 'not_cyberbullying']
gender_bully_df = bully_df[bully_df['type'] == 'gender']
religion_bully_df = bully_df[bully_df['type'] == 'religion']
age_bully_df = bully_df[bully_df['type'] == 'age']
ethnicity_bully_df = bully_df[bully_df['type'] == 'ethnicity']
other_bully_df = bully_df[bully_df['type'] == 'other_cyberbullying']
all_bully_df = bully_df[bully_df['type'] != 'not_cyberbullying']

print(not_bully_df.shape)
print(gender_bully_df.shape)
print(religion_bully_df.shape)
print(age_bully_df.shape)
print(ethnicity_bully_df.shape)
print(other_bully_df.shape)
print(all_bully_df.shape)

In [ ]:
not_bully_df.tail()

In [ ]:
sampled_not_bully_df = not_bully_df.sample(n=7000, random_state=42)
sampled_gender_bully_df = gender_bully_df.sample(n=1750, random_state=42)
sampled_age_bully_df = age_bully_df.sample(n=1750, random_state=42)
sampled_religion_bully_df = religion_bully_df.sample(n=1750, random_state=42)
sampled_ethnicity_bully_df = ethnicity_bully_df.sample(n=1750, random_state=42)
sampled_other_bully_df = other_bully_df.sample(n=1200, random_state=42)

sampled2_not_bully_df = not_bully_df.sample(n=3000, random_state=42)
sampled2_gender_bully_df = gender_bully_df.sample(n=600, random_state=42)
sampled2_age_bully_df = age_bully_df.sample(n=600, random_state=42)
sampled2_religion_bully_df = religion_bully_df.sample(n=600, random_state=42)
sampled2_ethnicity_bully_df = ethnicity_bully_df.sample(n=600, random_state=42)
sampled2_other_bully_df = other_bully_df.sample(n=600, random_state=42)

In [ ]:
sampled_gender_bully_df.shape

In [ ]:
sampled_gender_bully_df.tail()

In [ ]:
final_df = pd.concat([sampled_not_bully_df, sampled_gender_bully_df, sampled_age_bully_df, sampled_religion_bully_df, sampled_ethnicity_bully_df], ignore_index=True)
# final_df['tweet'] = final_df['tweetsplit'].apply(lambda x: ([w for w in x if len(w)>=3]))

In [ ]:
final2_df = pd.concat([sampled2_not_bully_df, sampled2_gender_bully_df, sampled2_age_bully_df, sampled2_religion_bully_df, sampled2_ethnicity_bully_df, sampled2_other_bully_df], ignore_index=True)
# final2_df['tweet'] = final2_df['tweetsplit'].apply(lambda x: ([w for w in x if len(w)>=3]))

In [ ]:
final_df.tail(20)

In [ ]:
final2_df.loc[final2_df['type']!='not_cyberbullying','type'] = 'yes'
final2_df.loc[final2_df['type']=='not_cyberbullying','type'] = 'no'

In [ ]:
# def allign(text):
#   while len(text)<5:
#     text.append('<oov>')
#   return text

# final_df['tweet'] = final_df['tweet'].apply(allign)
# final2_df['tweet'] = final2_df['tweet'].apply(allign)

In [ ]:
mapping = {'not_cyberbullying': 0, 'gender': 1, 'age': 1, 'ethnicity': 1, 'religion':1, 'other_cyberbullying':1}
mapping2 = {'yes': 1, 'no': 0}

In [ ]:
final_df.tail(20)

In [ ]:
final_df['type'] = final_df['type'].map(mapping)
final2_df['type'] = final2_df['type'].map(mapping2)

In [ ]:
final_df.tail(20)

In [ ]:
final_df.shape

In [ ]:
final_df.tail(20)

In [ ]:
final_df.shape

In [ ]:
X=final_df['tweet']
y=final_df['type']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15, random_state=42)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
max_len=120
X_train_encoded = tokenizer.batch_encode_plus(X_train.tolist(),
                                              padding=True,
                                              truncation=True,
                                              max_length = max_len,
                                              return_tensors='tf')

X_val_encoded = tokenizer.batch_encode_plus(X_val.tolist(),
                                              padding=True,
                                              truncation=True,
                                              max_length = max_len,
                                              return_tensors='tf')

X_test_encoded = tokenizer.batch_encode_plus(X_test.tolist(),
                                              padding=True,
                                              truncation=True,
                                              max_length = max_len,
                                              return_tensors='tf')

In [ ]:
X_train
# y[0]

In [ ]:
X_train_encoded['input_ids'][0]

In [ ]:
tokenizer.decode(X_train_encoded['input_ids'][0])

In [ ]:
X_train_encoded['attention_mask'][0]

In [ ]:
from transformers import TFBertForSequenceClassification
import tensorflow as tf
# recommended learning rate for Adam 5e-5, 3e-5, 2e-5
learning_rate = 2e-5
# we will do just 1 epoch, though multiple epochs might be better as long as we will not overfit the model
number_of_epochs = 1
# model initialization
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2,)
# The number of epochs is set to 2 as higher epochs will give rise to overfitting problems as well as take more time for the model to train.
# choosing Adam optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=1e-08)
# we do not have one-hot vectors, we can use sparce categorical cross entropy and accuracy
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [ ]:


callbacks = [EarlyStopping(monitor='val_loss')]
history = model.fit(
    [X_train_encoded['input_ids'], X_train_encoded['token_type_ids'], X_train_encoded['attention_mask']],
    y_train,
    validation_data=(
      [X_val_encoded['input_ids'], X_val_encoded['token_type_ids'], X_val_encoded['attention_mask']],y_val),
    batch_size=50,
    epochs=3,
    callbacks=callbacks
)

In [ ]:
plt.suptitle('Optimizer : Adam', fontsize=10)
plt.ylabel('Loss', fontsize=16)
plt.xlabel('Epoch', fontsize=14)
plt.plot(history.history['loss'], color='b', label='Training Loss')
plt.plot(history.history['val_loss'], color='r', label='Validation Loss')
plt.legend(loc='upper right')

In [ ]:
y_pred = model.predict([X_test_encoded['input_ids'], X_test_encoded['token_type_ids'], X_test_encoded['attention_mask']])
y_pred

In [ ]:
yy_pred=[]
for xx in y_pred.logits:
  yy_pred.append(np.argmax(xx))


In [ ]:
cnf_matrix = metrics.confusion_matrix(y_test, yy_pred)
cnf_matrix

In [ ]:
class_names=['no_bully', 'bully'] # name  of classes
fig, ax = plt.subplots(figsize=(3,3))

# create heatmap
sns.heatmap(cnf_matrix, annot=True, cmap="YlGnBu", fmt='g', cbar=False)
ax.set_xticklabels(class_names, rotation=45, ha='right')  # Rotate x-ticks for readability
ax.set_yticklabels(class_names, rotation=0)
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Confusion matrix', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label(bert)')

In [ ]:
target_names = ['no_bully', 'bully']
print(classification_report(y_test, yy_pred, target_names=target_names))

In [ ]:
y_score = y_pred.logits
print(y_score)

plt.figure(figsize=(10, 8))
y_test_one_hot = []
a1=[]
a1.append(1)
a1.append(0)
a2=[]
a2.append(0)
a2.append(1)
for x in y_test:
  if x==0:
    y_test_one_hot.append([1,0])
  else:
    y_test_one_hot.append([0,1])
yt1=np.array(y_test_one_hot)
print(yt1[:, 0])
yt2=np.array(y_score)

for i in range(2):
    fpr, tpr, _ = roc_curve(yt1[:, i], yt2[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f'Class {i} (AUC = {roc_auc:.2f})')

plt.plot([0, 1], [0, 1], 'k--', label='Random Guessing')

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for Each Class')
plt.legend(loc='best')
plt.show()

In [ ]:
logacc = accuracy_score(y_test, yy_pred)
print(f"Model Accuracy: {logacc:.2f}")